### Dask DataFrames

Similar to the design of arrays, dask dataframes are paritioned pandas dataframes. Computations run pandas operators on the chunks and aggregate results from the chunked operations.

<img src="https://upload.wikimedia.org/wikipedia/commons/8/81/Dask-dataframe.svg" width="256" title="    https://upload.wikimedia.org/wikipedia/commons/8/81/Dask-dataframe.svg" />

For the most part, dask has tried to implement all of pandas, but there are some inefficient operations that it does not support.

Let's load up a dataframe and see what we get.  This is the NYC flight data used in the dask tutorial.

In [5]:
import dask.dataframe as dd
df = dd.read_csv('./data/nycflight/*.csv',
                 dtype={'TailNum': str,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})
df

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
npartitions=10,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int64,int64,float64,int64,float64,int64,string,int64,string,float64,float64,float64,float64,float64,string,string,float64,float64,float64,bool,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
df.tail()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
269176,1999,12,27,1,1645.0,1645,1830.0,1901,UA,1753,...,205.0,-31.0,0.0,LGA,DEN,1619.0,7.0,13.0,False,0
269177,1999,12,28,2,1726.0,1645,1928.0,1901,UA,1753,...,214.0,27.0,41.0,LGA,DEN,1619.0,5.0,23.0,False,0
269178,1999,12,29,3,1646.0,1645,1846.0,1901,UA,1753,...,220.0,-15.0,1.0,LGA,DEN,1619.0,5.0,15.0,False,0
269179,1999,12,30,4,1651.0,1645,1908.0,1901,UA,1753,...,233.0,7.0,6.0,LGA,DEN,1619.0,5.0,19.0,False,0
269180,1999,12,31,5,1642.0,1645,1851.0,1901,UA,1753,...,232.0,-10.0,-3.0,LGA,DEN,1619.0,6.0,11.0,False,0


In [7]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
0,1990,1,1,1,1621.0,1540,1747.0,1701,US,33,...,NaN,46.0,41.0,EWR,PIT,319.0,NaN,NaN,False,0
1,1990,1,2,2,1547.0,1540,1700.0,1701,US,33,...,NaN,-1.0,7.0,EWR,PIT,319.0,NaN,NaN,False,0
2,1990,1,3,3,1546.0,1540,1710.0,1701,US,33,...,NaN,9.0,6.0,EWR,PIT,319.0,NaN,NaN,False,0
3,1990,1,4,4,1542.0,1540,1710.0,1701,US,33,...,NaN,9.0,2.0,EWR,PIT,319.0,NaN,NaN,False,0
4,1990,1,5,5,1549.0,1540,1706.0,1701,US,33,...,NaN,5.0,9.0,EWR,PIT,319.0,NaN,NaN,False,0


The dataframe has 10 chunks (npartitions) that correspond to the ten files that were read.  Dataframes have two key properties:
  * they are tabular (two-dimensional) data
  * each column has a datatype defined by a _schema_
  
The programmer is encouraged to think of this as "spreadsheet or SQL table".  It is reasonable to call the data __structured__.
  * this is in contrast to semi-structured data which has tags and hierarchy, but does not enforce types. Examples are XML and JSON.
  * in database parlance, this is flat data model.

### Data Slicing and Aggregation

The most basic operations in a database and in dask is to <code>select</code> rows and <code>project</code> columns.

#### Selecting rows

Find all flights flown by a specific plane, identified by <code>TailNum</code>

In [8]:
df2 = df[df.TailNum=='N516UA'].compute()
df2

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
2295,1995,1,26,4,1822.0,1815,2147.0,2129,UA,71,...,359.0,18.0,7.0,EWR,SFO,2565.0,7.0,19.0,False,0
2304,1995,1,4,3,1757.0,1715,2103.0,2029,UA,73,...,337.0,34.0,42.0,EWR,SFO,2565.0,6.0,23.0,False,0
2349,1995,1,18,3,1049.0,1050,1348.0,1404,UA,75,...,330.0,-16.0,-1.0,EWR,SFO,2565.0,4.0,25.0,False,0
2351,1995,1,20,5,1119.0,1050,1416.0,1404,UA,75,...,337.0,12.0,29.0,EWR,SFO,2565.0,10.0,10.0,False,0
2369,1995,1,7,6,700.0,700,1041.0,1019,UA,79,...,368.0,22.0,0.0,EWR,SFO,2565.0,7.0,26.0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248678,1999,12,16,4,701.0,700,1019.0,1024,UA,853,...,337.0,-5.0,1.0,EWR,SFO,2565.0,22.0,19.0,False,0
248680,1999,12,18,6,659.0,700,1033.0,1024,UA,853,...,363.0,9.0,-1.0,EWR,SFO,2565.0,19.0,12.0,False,0
269006,1999,12,5,7,820.0,820,1024.0,1043,UA,329,...,214.0,-19.0,0.0,LGA,DEN,1619.0,14.0,16.0,False,0
269094,1999,12,1,3,616.0,620,814.0,832,UA,875,...,215.0,-18.0,-4.0,LGA,DEN,1619.0,7.0,16.0,False,0


#### Projecting Columns

Build a dataframe that describes the plane ('TailNum') and route ('FlightNum')

In [32]:
routes = df[['FlightNum','TailNum']].compute()

routes

,FlightNum,TailNum
0,33,NaN
1,33,NaN
2,33,NaN
3,33,NaN
4,33,NaN
...,...,...
269176,1753,N516UA
269177,1753,N504UA
269178,1753,N592UA
269179,1753,N575UA


### Aggregating Data

A common data science inquiry is to query an aggregate (mean, min, max, sum, etc.) in a group.  This is done with a <code>groupby</code> query. The pattern is to construct a grouping and then aggregate over the grouping.

__Query:__ How many different flights were flown by each plane?

In [21]:
#routes.groupby('TailNum')['FlightNum'].nunique()
routes.groupby('TailNum').FlightNum.nunique()

TailNum
'144DA       1
112          4
A367NW      30
EI-BWD     122
EI-CAL      10
          ... 
NEIDLA      16
NN7324      40
NOZ1AA       1
NXXXXX       4
UNKNOW    1463
Name: FlightNum, Length: 3712, dtype: int64

I have intentionaly mixed syntax. Dataframes in both R and Python use two form of syntax interchangeably.
  * <code>dataframe.columnName</code>
  * <code>dataframe['columnName']

Many functions only accept the bracketed indexing of columns.
    
__Query:__ How many times was each flight flown?

In [48]:
routes.groupby('FlightNum').FlightNum.count()

FlightNum
1       5399
2         29
3       7203
4        166
5       7225
        ... 
9607       1
9608       1
9777       2
9851       1
9899       2
Name: FlightNum, Length: 2405, dtype: int64

__Query:__ What were the most routes flown by a single plane?

In [49]:
routes.groupby('TailNum')['FlightNum'].nunique().max()

1463

__Query:__ What is the maximum number of planes to fly a single route?

In [51]:
routes.groupby('FlightNum')['TailNum'].nunique().max()

1045

But, these aggregates are not really the questions we want answered. More natural questions are awkward.
  * What route was flown by the most planes?
  * What plane flew the most routes?

In [57]:
routes.groupby('FlightNum').TailNum.nunique().idxmax()

305

In [58]:
routes.groupby('TailNum').FlightNum.nunique().idxmax()

'UNKNOW'

This reveals problems with the data.  So, let's look for an actual plane.

In [59]:
routes[routes.TailNum != 'UNKNOW'].groupby('TailNum').FlightNum.nunique().idxmax()

'N16301'

and ask for how many different flights it has flown

In [70]:
routes[routes.TailNum != 'UNKNOW'].groupby('TailNum').FlightNum.nunique().max()

442

In [ ]:
routes[routes.TailNum != 'UNKNOW'].groupby('TailNum').FlightNum.nunique().max()

and verify that this is the right answer.

In [71]:
routes[routes.TailNum == 'N16301'].FlightNum.nunique()

442

### Indexes

All dataframes have a __default index__.  In this case, the index was generated when we loaded the data and is the row number in the pandas dataframe. Surprisingly, the index is not unique.  The same index value appears in each pandas dataframe.

In [ ]:
print(df.index, "\n")
print("Number of rows in the database\n", len(df))
maxindex = df.index.nunique().compute()
print("Number of unique values in the index\n", maxindex)

# find all entries with index value 22000
df.loc[22000].compute()

In [ ]:
df.index.head()

The term index is used to mean many things in computer science:
* the offset of an element in a data structure, i.e. the array index
* a data structure used for fast access in a data store

When I heard index in dask, I expected it to be the latter. It is not. By default, it is only the row number in the local pandas data frame.


My concept of an index comes from relational databases: "A database index is a data structure that improves the speed of data retrieval operations on a database table at the cost of additional writes and storage space to maintain the index data structure."

Real indexes come in many forms, but are most commonly:

* hash tables -- organize the data by the hash value of a key field for constant time $O(1)$ lookup by key.

<img src="https://uploads-ssl.webflow.com/632c5ceb41133222d120b86e/632f1efd49442a2371a6bb8a_hash-function.png" width=256 />

* B+-trees/sorted -- sort the data in a tree to lookup a key in $O(\log n)$ time and be able to scan sequential keys.

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*brGYM6uwjUmAaGLs1QKKxw.png" width=386 />

This is not really a topic for this class. Although, indexing is one of my favorite parts of computer science.

A **simulacrum** of indexing can be accomplished in dask by calling <code>set_index</code>, which often results in a global shuffle of the data.  It can be very expensive. After an index is set, all of the data are sorted (across all partitions). This makes it much more efficient to conduct some database like operations, `join()`, `groupby()`, etc.

### Why do indexes matter?

Understanding the structure of the can lead to massive performance differences.
We can create an example in which seemingly identical queries perform 
differently because of an implicit index structure.

We create 100 files each with 100000 entries with two fields 'A' and 'B'.  'A' contains an integer that identifies the file number (0-99). 'A' is the same in each file. 'B' contains a sequence of numbers 0-1000000 in each file. These data:
  * are too big to fit in memory
  * have one field that will be identical in each pandas dataframe

To generate the data, run the cell **at the end** of the workbook once.

In [8]:
df = dd.read_csv('/tmp/csv*.csv')
df.head
df.compute()

,A,B
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
499995,99,499995
499996,99,499996
499997,99,499997
499998,99,499998


Now let's compare how long it takes to sum all the elements grouped by each key for both columns.  In both cases, we touch all of the data.  But, the sums in when we <code>groupby</code> 'A' can all be evaluated in one chunk.  When we <code>groupby</code> 'B', partial sums at each value must be aggregated across all chunks.

In [9]:
%time df.groupby('A').sum().compute()

CPU times: user 4.93 s, sys: 402 ms, total: 5.33 s
Wall time: 1.75 s


,B
A,
74,124999750000
52,124999750000
88,124999750000
93,124999750000
97,124999750000
...,...
9,124999750000
4,124999750000
44,124999750000


In [10]:
%time df.groupby('B').sum().compute()

CPU times: user 17.9 s, sys: 4.78 s, total: 22.7 s
Wall time: 6.55 s


,A
B,
52,4950
74,4950
88,4950
93,4950
97,4950
...,...
499955,4950
499961,4950
499963,4950


A couple of observations:
- Dask is parallelizing the computation across multiple nodes.
  - notice the difference between wall time and CPU times.
- summing by `B` is much less efficient
  - it has interference between different parallel chunks
  - sending the data from node to node
  
Now let's set up an index and compare:

In [11]:
df = dd.read_csv('/tmp/csv*.csv')
%time df = df.set_index('A').compute()
df

CPU times: user 15.5 s, sys: 3.33 s, total: 18.9 s
Wall time: 6.49 s


,B
A,
0,0
0,1
0,2
0,3
0,4
...,...
99,499995
99,499996
99,499997


In [12]:
%time df.groupby('A').sum()

CPU times: user 401 ms, sys: 457 ms, total: 858 ms
Wall time: 853 ms


,B
A,
0,124999750000
1,124999750000
2,124999750000
3,124999750000
4,124999750000
...,...
95,124999750000
96,124999750000
97,124999750000


In [2]:
df = dd.read_csv('/tmp/csv*.csv')
%time df = df.set_index('B').compute()
df

CPU times: user 25.4 s, sys: 8.39 s, total: 33.8 s
Wall time: 16.9 s


,A
B,
0,71
0,67
0,42
0,27
0,23
...,...
499999,25
499999,75
499999,17


In [3]:
%time df.groupby('B').sum()

CPU times: user 489 ms, sys: 194 ms, total: 682 ms
Wall time: 678 ms


,A
B,
0,4950
1,4950
2,4950
3,4950
4,4950
...,...
499995,4950
499996,4950
499997,4950


We observe that computing aggregates or building indexes within partitions `A` is faster than across partitions `B`, owing to interference

It only makes sense to build an index if you are going to use it multiple times. In this way, you invest in computing once and benefit over multiple invocations.

Dask has implemented some what I would expect out of indexing:
1. dask can be told that there is existing structure in the data
2. dask reorganizes data to optimize queries

Dask does not support some features that I think it should
1. hash indexing
2. data readers infer the existence of an index

Why is this a simulacrum of an index, and not a real database structure. Because it only sorts the data, if does not build an additional data structure for fast access. The improved performance only comes from the sortedness of the data, e.g.:
* find an element using the index field can use binary search, rather than a scan
* joining data on the index field can be done using a "two-finger join" in a linear scan of two tables without sorting (data are presorted)

### Parting Thoughts

Limitations and comments:
  * dask does not have a general sort capability
    * but, can be accomplished with set_index
    * shuffle is inefficient, use a different engine

### Generate data

Uncomment and run this once to make data.

In [1]:
import csv
for i in range(100):
    with open(f'/tmp/csv{i}.csv', 'w', newline='') as csvfile:
        csvw = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvw.writerow(['A','B'])
        for j in range(500000):
            csvw.writerow([i,j])